In [22]:
## GPT를 활용해서 Few shot inference를 수행합니다.
## 하나의 data 생성에 30초정도 걸립니다... korquad의 unique한 context 9000개를 활용해서 
## 문제를 제작할 경우 약 16시간정도의 시간이 걸립니다.
import openai
from datasets import load_dataset
import pandas as pd
import numpy as np
from ast import literal_eval

client = openai.OpenAI(
    api_key="your api key"
)


In [ ]:
import unicodedata

def normalize_unicode(text):
    # 유니코드 문자열을 ASCII로 정규화
    return unicodedata.normalize("NFKD", text)

def generate(row):
    choices = row['choices']  # 이미 전처리된 'choices' 사용
    # 선택지 텍스트 정리
    choices_text = "\n".join([f"{idx + 1}. {choice}" for idx, choice in enumerate(choices)])  # 선택지 정리

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 문제의 해설을 작성하는 수능 출제자입니다.
                               지문, 질문, 선택지를 읽고 각 선택지에 대한 해설을 작성하세요.
                               반드시 해설은 200자 이내로 작성하세요.
                               반드시 선택지 N: 해설 형식을 사용하세요.
                               """,
            },
            {
                "role": "user",
                "content": f""" 지문 : {row['paragraph']}
                                질문 : {row['question']}
                                선택지 : {choices_text}
                                정답 : {row['answer']}
                                해설 : [model output]
                            """
            },
        ],
    )
    response = response.choices[0].message.content.strip()
    row = {
        "id": f"yongari_generate_{int(str(abs(hash(row['paragraph'])))[:5])}",
        "paragraph": row['paragraph'],
        "problems": row['problems'],
        "klue" : response,
        "question_plus": np.nan,

    }
    return row


In [ ]:
data = pd.read_csv('data/train.csv')
data

,id,paragraph,problems,question_plus
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",{'question': '상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두...,NaN
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...","{'question': '(가)에 대한 설명으로 옳지 않은 것은?', 'choice...",NaN
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,"{'question': '(가) 지역에 대한 설명으로 옳은 것은?', 'choice...",NaN
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,"{'question': '밑줄 친 ‘그’에 대한 설명으로 옳은 것은?', 'choi...",NaN
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...","{'question': '(가) 인물이 추진한 정책으로 옳지 않은 것은?', 'ch...",NaN
...,...,...,...,...
2026,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...","{'question': '잭 맥도널드가 남긴 유산의 총액은 얼마인가?', 'choi...",NaN
2027,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...","{'question': ""넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇...",NaN
2028,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,{'question': '옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇...,NaN
2029,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,{'question': '방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?'...,NaN


In [ ]:
from tqdm import tqdm
from ast import literal_eval

data['dic'] = data['problems'].apply(lambda x: literal_eval(x))
data['question'] = data['dic'].apply(lambda x: x['question'])
data['answer'] = data['dic'].apply(lambda x: x['answer'])
data['choices'] = data['dic'].apply(lambda x: x['choices'])

generated = []
for idx, row in tqdm(data.iterrows()):
    generated.append(generate(row))

generated = pd.DataFrame(generated)

2031it [59:43,  1.76s/it]


In [ ]:
generated.to_csv('train_why.csv', index = False)